# Görüntü İşleme Nesne Tespiti Projesi

### Barış Onur Uzgur - 161024065

In [1]:
#gerekli kütüphaneler eklendi
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

In [2]:
#Kontur bulma işleminden sonra kırpıcağımız kontur alanı en buyuk kontur olacaktır.Bu fonksiyonla konturler arasındaki
# en büyük alanlı kontoru buluruz.
def biggest_contour(contours,image_size):#parametreler bulunan konturler ile goruntunun alanı(örn. 1920*1080=...)
    #ihtiyacımız olan boş değişkenler tanımlandı
    biggest = None
    max_area = 0
    biggest_n=0
    approx_contour=None
    for n,i in enumerate(contours):# burdaki enumerate conturler arasındaki sıralama işlemini yapmaktadır. 
            area = cv.contourArea(i)#conturlerin alanınını bulduk
         
            
            if area > image_size/10:#kendi belirlediğimiz oranla kontur alanını kıyasladık.
                    peri = cv.arcLength(i,True)
                    approx = cv.approxPolyDP(i,0.02*peri,True)#hata oranını %2 olarak belirledik.
                    if area > max_area and len(approx)==4:
                            biggest = approx#gerekli atamalar yapıldı.
                            max_area = area
                            biggest_n=n
                            approx_contour=approx
                            
                                                   
    return biggest_n,approx_contour

In [3]:
# bu fonksiyon en büyük konturun koordinatları hepsalayıp bir dikdörtgen olarak çıktı verir.
def order_points(pts):#Burdaki paramatre en büyük alanlı konturun köşegenlerinin kordinatlarıdır.
    pts=pts.reshape(4,2)#kordinatların şekli (4,1,2) olduğu için yeniden şekillendirildi.
    rect = np.zeros((4, 2), dtype = "float32")#boş deişken deklare edildi.

    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

  
    return rect

In [4]:
#bu fonksiyon orjinal görüntüden koordinatları bulunan konturu kırpar.
def four_point_transform(image, pts):#Parametreler görüntü ve kontur koordinalarıdır.
    
    rect = order_points(pts)#Önceden yazılmış fonksiyon çağrıldı.
    (tl, tr, br, bl) = rect#dikdörtgen tanımlandı.

    #Gereki işlemler yapıldı.
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))
   

    
    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    M = cv.getPerspectiveTransform(rect, dst)
    warped = cv.warpPerspective(image, M, (maxWidth, maxHeight))

    
    return warped

In [5]:
def nesnetespiti(img):
    kernel = np.ones((5,5))#Morfolojik işlemler için kernel deklare ettik.
    image=img.copy()  #görüntüyü kopyaladık.
    height, width, channels = image.shape#görüntünün boyutlarını aldık.
    img_gray=cv.cvtColor(image,cv.COLOR_BGR2GRAY)#görüntüyü gri seviyeye dönüştürdük.
    image_size=img_gray.size
    

    blur = cv.blur(img_gray,(5,5),0)#Görüntüyü bulanıklaştırdık.
    
   
    th3 = cv.adaptiveThreshold(blur,255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY,11,2)# Adaptive Gaussian Threshold
   
    
    
    denoising = cv.fastNlMeansDenoising(th3, 11, 31, 9)#Görüntüde gürültü azaltmaya yarayan fonksiyon.
  
    
    erosion = cv.erode(denoising,kernel,iterations = 1)#erosion işlemini gerçekleştiren fonksiyon.
    
    
    closing = cv.morphologyEx(erosion, cv.MORPH_CLOSE, kernel)#closing işlemini gerçekleştiren fonksiyon.
    
    
    edges = cv.Canny(closing,50,150,apertureSize = 7)# Canny Edge Detection gerçekleştirildi.
   
   
    contours, hierarchy = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)#kontürler bulundu.
    simplified_contours = []#sadeleştirilmiş konturler için boş bir değişken deklare edildi.

    #konturleri sadeleştirmek için bir döngü oluşturduk.
    for cnt in contours:
        hull = cv.convexHull(cnt)#konturleri dışbükey olarak birleştirme fonksiyonu
        simplified_contours.append(cv.approxPolyDP(hull,
                                0.001*cv.arcLength(hull,True),True))
    simplified_contours = np.array(simplified_contours)
    
    biggest_n,approx_contour = biggest_contour(simplified_contours,image_size)#Daha önce yazılan en büyük kontur bulma 
                                                                              #fonksiyonu kullanıldı.

    threshold = cv.drawContours(image, simplified_contours ,biggest_n, (0,255,0), 1)#konturler bastırıldı.
  
    dst = 0
    if approx_contour is not None and len(approx_contour)==4:
        approx_contour=np.float32(approx_contour)
        dst=four_point_transform(threshold,approx_contour)
    
    cropped = dst
    
    
    
    return cropped

In [8]:
def iyilestirme(img):#görüntüyü iyileştirmek için bir fonksiyon yazıldı.
    filter = np.array([[-1,-1,-1],[-1,9,-1],[-1,-1,-1]])#keskinleştirme maskesi tanımlandı
    sharpened = cv.filter2D(img,-1,filter)#Filtreleme işlemi
    
    return sharpened

In [ ]:
def scanner(img):#iki ana fonksiyon kullanılak tek fonksiyon kullanıldı.
    warped = nesnetespiti(img)
    sharpened=iyilestirme(warped)
    return sharpened

In [7]:
#main kod
#Bu cellde inputlar alınalarak yazılan fonksiyonlarla işlenip bastırıldı.
egitim1  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Eğitim\egitim1.jpeg"  )  
egitim2  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Eğitim\egitim2.jpeg"  )  
egitim3  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Eğitim\egitim3.jpeg"  )  
egitim4  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Eğitim\egitim4.jpeg"  )  

test1  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Test\test1.jpeg"  )  
test2  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Test\test1.jpeg"  )  
test3  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Test\test1.jpeg"  )  
test4  =  cv . imread ("C:\Users\barisonur.uzgur2016\Desktop\Görüntü Proje\Görüntüler\Test\test1.jpeg" )  

egitim1_out = scanner(egitim1)
egitim2_out = scanner(egitim2)
egitim3_out = scanner(egitim3)
egitim4_out = scanner(egitim4)

test1_out = scanner(test1)
test2_out = scanner(test2)
test3_out = scanner(test3)
test4_out = scanner(test4)

cv.imwrite("egitim1_out.jpeg", egitim1_out)
cv.imwrite("egitim2_out.jpeg", egitim2_out)
cv.imwrite("egitim3_out.jpeg", egitim3_out)
cv.imwrite("egitim4_out.jpeg", egitim4_out)


cv.imwrite("test1_out.jpeg", test1_out)
cv.imwrite("test2_out.jpeg", test2_out)
cv.imwrite("test3_out.jpeg", test3_out)
cv.imwrite("test4_out.jpeg", test4_out)
